In [1]:
import tensorflow as tf

In [16]:
FOV = 115
OUTPT = 151
INPT = OUTPT + FOV - 1


map_0 = 1

# Layer 1: Diluted Convolution
field_1 = 4
stride_1 = 1
dilation_1 = 1
map_1 = 48

image = tf.placeholder(tf.float32, shape=[1, 95, 95, 1])
# Shape: [# layers per batch, patch size, patch size, labels per pixel]
target = tf.placeholder(tf.float32, shape=[None, None, None, 1])

In [17]:
W_conv1 = tf.Variable(tf.truncated_normal([field_1, field_1, map_0, map_1], stddev=0.1))
b_conv1 = tf.Variable(tf.constant(0.1, shape=[map_1]))
conv1 = tf.nn.convolution(image, W_conv1, strides=[stride_1, stride_1], padding='VALID',
                          dilation_rate=[dilation_1, dilation_1])
h_conv1 = tf.nn.relu(conv1 + b_conv1)

In [18]:
print(W_conv1.get_shape())
print(b_conv1.get_shape())
print(conv1.get_shape())
print(h_conv1.get_shape())

(4, 4, 1, 48)
(48,)
(1, 92, 92, 48)
(1, 92, 92, 48)


In [21]:
field_2 = 2
stride_2 = 1
dilation_2 = 1
h_pool1 = tf.nn.pool(h_conv1, window_shape=[field_2, field_2], dilation_rate=[dilation_2, dilation_2],
                             strides=[stride_2, stride_2], padding='VALID', pooling_type='MAX')

In [22]:
print(h_pool1.get_shape())

(1, 91, 91, 48)


In [23]:
def weight_variable(shape):
  """
  One should generally initialize weights with a small amount of noise
  for symmetry breaking, and to prevent 0 gradients.
  Since we're using ReLU neurons, it is also good practice to initialize
  them with a slightly positive initial bias to avoid "dead neurons".
  """
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)


def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)


def conv2d(x, W, dilation=None):
  return tf.nn.convolution(x, W, strides=[1, 1], padding='VALID', dilation_rate=[dilation, dilation])


def max_pool(x, dilation=None, strides=[2, 2], window_shape=[2, 2]):
  return tf.nn.pool(x, window_shape=window_shape, dilation_rate= [dilation, dilation],
                       strides=strides, padding='VALID', pooling_type='MAX')

In [51]:
FOV = 95
OUTPT = 101
INPT = FOV + 2 * (OUTPT//2)


params = {
    'm1': 48,
    'm2': 48,
    'm3': 48,
    'm4': 48,
    'fc': 200,
    'lr': 0.001,
    'out': 101
}

map_1 = params['m1']
map_2 = params['m2']
map_3 = params['m3']
map_4 = params['m4']
fc = params['fc']
        
image = tf.placeholder(tf.float32, shape=[1, INPT, INPT, 1])
target = tf.placeholder(tf.float32, shape=[1, OUTPT, OUTPT, 2])

# layer 1 - original stride 1
W_conv1 = weight_variable([4, 4, 1, map_1])
b_conv1 = bias_variable([map_1])
h_conv1 = tf.nn.relu(conv2d(image, W_conv1, dilation=1) + b_conv1)

# layer 2 - original stride 2
h_pool1 = max_pool(h_conv1, strides=[1, 1], dilation=1)

# layer 3 - original stride 1
W_conv2 = weight_variable([5, 5, map_1, map_2])
b_conv2 = bias_variable([map_2])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, dilation=2) + b_conv2)

# layer 4 - original stride 2
h_pool2 = max_pool(h_conv2, strides=[1, 1], dilation=2)

# layer 5 - original stride 1
W_conv3 = weight_variable([4, 4, map_2, map_3])
b_conv3 = bias_variable([map_3])
h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3, dilation=4) + b_conv3)

# layer 6 - original stride 2
h_pool3 = max_pool(h_conv3, strides=[1, 1], dilation=4)

# layer 7 - original stride 1
W_conv4 = weight_variable([4, 4, map_3, map_4])
b_conv4 = bias_variable([map_4])
h_conv4 = tf.nn.relu(conv2d(h_pool3, W_conv4, dilation=8) + b_conv4)

# layer 8 - original stride 2
h_pool4 = max_pool(h_conv4, strides=[1, 1], dilation=8)

# layer 9 - original stride 1
W_fc1 = weight_variable([3, 3, map_4, fc])
b_fc1 = bias_variable([fc])
h_fc1 = tf.nn.relu(conv2d(h_pool4, W_fc1, dilation=16) + b_fc1)

# layer 10 - original stride 2
W_fc2 = weight_variable([1, 1, fc, 2])
b_fc2 = bias_variable([1])
prediction = conv2d(h_fc1, W_fc2, dilation=16) + b_fc2

s_pred = tf.nn.sigmoid(prediction)

In [34]:
print("L1")
print(h_conv1.get_shape())
print("L2")
print(h_pool1.get_shape())
print("L3")
print(h_conv2.get_shape())
print("L4")
print(h_pool2.get_shape())
print("L5")
print(h_conv3.get_shape())
print("L6")
print(h_pool3.get_shape())
print("L7")
print(h_conv4.get_shape())
print("L8")
print(h_pool4.get_shape())
print("L9")
print(h_fc1.get_shape())
print("L10")
print(prediction.get_shape())
print("Pred")
print(s_pred.get_shape())

L1
(1, 192, 192, 48)
L2
(1, 191, 191, 48)
L3
(1, 183, 183, 48)
L4
(1, 181, 181, 48)
L5
(1, 169, 169, 48)
L6
(1, 165, 165, 48)
L7
(1, 141, 141, 48)
L8
(1, 133, 133, 48)
L9
(1, 101, 101, 200)
L10
(1, 101, 101, 2)
Pred
(1, 101, 101, 2)


In [58]:
%load_ext autoreload
%autoreload 2
import models

In [62]:
import models.conv_net as cnet

In [83]:
cnet.ConvNet(cnet.DEFAULT_PARAMS)

Initial Layer
(1, 195, 195, 1)
Layer 1,	type: conv2d,	filter: [4, 4],	FOV: 4
Layer 2,	type: pool,	filter: [2, 2],	FOV: 5
Layer 3,	type: conv2d,	filter: [5, 5],	FOV: 13
Layer 4,	type: pool,	filter: [2, 2],	FOV: 15
Layer 5,	type: conv2d,	filter: [4, 4],	FOV: 27
Layer 6,	type: pool,	filter: [2, 2],	FOV: 31
Layer 7,	type: conv2d,	filter: [4, 4],	FOV: 55
Layer 8,	type: pool,	filter: [2, 2],	FOV: 63
Layer 9,	type: conv2d,	filter: [3, 3],	FOV: 95
Layer 10,	type: conv2d,	filter: [1, 1],	FOV: 95


<models.conv_net.ConvNet instance at 0x7f606d36c5f0>

50